<a href="https://colab.research.google.com/github/thisisbhupendrasingh/Transformers/blob/main/Transformers_on_CiFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformer Implementation on CiFAR-10 datasets

In [1]:
# Install required libraries
# !pip install transformers torch torchvision datasets

import torch
from torchvision import transforms
from transformers import ViTImageProcessor, ViTForImageClassification, TrainingArguments, Trainer
from huggingface_hub import login
from google.colab import userdata
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset

In [2]:
dataset = load_dataset('cifar10')
train_data = dataset["train"]
test_data = dataset["test"]

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/120M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/23.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [3]:
train_data[:5]  #samples

{'img': [<PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>],
 'label': [0, 6, 0, 2, 7]}

In [4]:
train_data.shape

(50000, 2)

In [5]:
class_labels = {0: "airplane", 1: "automobile", 2: "bird", 3: "cat", 4: "deer", 5: "dog", 6: "frog", 7: "horse", 8: "ship", 9: "truck"}

In [6]:
# Load Hugging Face token from Colab Secrets
hf_token = userdata.get('HF_TOKEN')
if hf_token:
    login(hf_token)
    print("Successfully logged in to Hugging Face!")
else:
    raise ValueError("HF_TOKEN not found in Colab Secrets. Please add it.")

# Load CIFAR-10 dataset
dataset = load_dataset('cifar10')
train_data = dataset["train"]
test_data = dataset["test"]
print("Train dataset size is: ",train_data.shape)
print("Test dataset size is: ",test_data.shape)

train_images = train_data["img"]
train_labels = train_data["label"]

test_images = test_data["img"]
test_labels = test_data["label"]

# Load pre-trained Vision Transformer and image processor
model_name = 'google/vit-base-patch16-224'
processor = ViTImageProcessor.from_pretrained(model_name, token=hf_token)
image_predictor_model = ViTForImageClassification.from_pretrained(model_name, token=hf_token)

Successfully logged in to Hugging Face!
Train dataset size is:  (50000, 2)
Test dataset size is:  (10000, 2)


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [7]:
# Define preprocessing transform
def transform(example):
    example["pixel_values"] = processor(images=example["img"], return_tensors="pt")["pixel_values"][0]
    return example

# Apply transform
dataset = dataset.with_transform(transform)
dataset

DatasetDict({
    train: Dataset({
        features: ['img', 'label'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['img', 'label'],
        num_rows: 10000
    })
})

In [8]:
labels = dataset["train"].features["label"].names
num_labels = len(labels)


model = ViTForImageClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)},
    token=hf_token,
    ignore_mismatched_sizes=True
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([10, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# !pip install -U transformers datasets evaluate

In [10]:
TrainingArguments?

In [11]:
training_args = TrainingArguments(
    output_dir="./vit-cifar10",
    do_eval=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    push_to_hub=False  # Set True if you want to upload to Hugging Face Hub
)

In [ ]:
# Load pre-trained Vision Transformer and image processor
model_name = 'google/vit-base-patch16-224'
processor = ViTImageProcessor.from_pretrained(model_name, token=hf_token)
image_predictor_model = ViTForImageClassification.from_pretrained(model_name, token=hf_token)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [ ]:
# processor(images=train_images, return_tensors="pt")

processor(images=train_images, return_tensors="pt")

ValueError: Could not make a flat list of images from Column([<PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32 at 0x7E4A2A8917F0>, <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32 at 0x7E4A2A892FC0>, <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32 at 0x7E4A2A890B30>, <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32 at 0x7E4A2A892960>, <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32 at 0x7E4A2A890050>])

In [ ]:
# images = dataset['img']  # Take 5 test images for simplicity
# labels = dataset['label']
# class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Load pre-trained Vision Transformer and image processor
model_name = 'google/vit-base-patch16-224'
processor = ViTImageProcessor.from_pretrained(model_name, token=hf_token)
model = ViTForImageClassification.from_pretrained(model_name, token=hf_token)

# Preprocess images
inputs = processor(images=images, return_tensors='pt')

# Predict
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1).numpy()

In [ ]:
outputs

ImageClassifierOutput(loss=None, logits=tensor([[ 1.3396, -0.2664, -0.6982,  ...,  0.2850,  1.2883,  0.1281],
        [ 3.0018,  0.5756, -0.0701,  ..., -1.1536, -0.8489, -0.1052],
        [-0.0189, -1.6661, -0.3012,  ..., -0.4646, -0.6377, -0.2056],
        [ 0.3447, -1.0012, -0.5074,  ..., -0.6837, -0.5289, -0.7313],
        [ 0.9016,  0.2513, -0.5265,  ...,  0.2486,  0.1768,  0.7395]]), hidden_states=None, attentions=None)

In [ ]:
# Display results
plt.figure(figsize=(15, 5))
for i, (img, pred, true) in enumerate(zip(images, predictions, labels)):
    plt.subplot(1, 5, i + 1)
    plt.imshow(img)
    plt.title(f"Pred: {class_names[pred]}\nTrue: {class_names[true]}")
    plt.axis('off')
plt.tight_layout()
plt.show()

# Print predictions
print("\nPrediction Results:")
for i, (pred, true) in enumerate(zip(predictions, labels)):
    print(f"Image {i+1}: Predicted = {class_names[pred]}, True = {class_names[true]}")